In [1]:
import torch 
import torch.nn.functional as F 
import matplotlib.pyplot as plt 
%matplotlib inline

In [2]:
words = open('names.txt','r') .read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [3]:
len(words)

32033

In [4]:
chars = sorted(list(set("".join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s, i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [10]:
block_size=3 
X,Y = [],[]
for w in words[:5]:
    print(w)
    
    context = [0]*block_size
    for ch in w +'.':
        # print(ch)
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        print(''.join(itos[i] for i in context), '--->', itos[ix] )
        context = context[1:] + [ix] #crop and append 

X = torch.tensor(X)
Y = torch.tensor(Y)

emma
... ---> e
..e ---> m
.em ---> m
emm ---> a
mma ---> .
olivia
... ---> o
..o ---> l
.ol ---> i
oli ---> v
liv ---> i
ivi ---> a
via ---> .
ava
... ---> a
..a ---> v
.av ---> a
ava ---> .
isabella
... ---> i
..i ---> s
.is ---> a
isa ---> b
sab ---> e
abe ---> l
bel ---> l
ell ---> a
lla ---> .
sophia
... ---> s
..s ---> o
.so ---> p
sop ---> h
oph ---> i
phi ---> a
hia ---> .


In [14]:
# converting 27 dimesntion into 2 dimension lookup sapce 
C = torch.randn((27,2))
C

tensor([[-0.2083, -0.4621],
        [-1.0806, -0.4659],
        [-0.2242, -0.6150],
        [ 1.1574, -0.5633],
        [ 0.7722,  1.3370],
        [ 0.5129, -0.5841],
        [ 0.0960,  0.2531],
        [-0.1081, -0.2200],
        [-0.4816,  1.5123],
        [-0.7295,  0.1149],
        [ 0.0805, -1.8379],
        [ 1.2840, -0.4023],
        [-1.1707,  0.2185],
        [ 0.1347, -0.5181],
        [ 0.4191,  0.4557],
        [ 0.4137,  0.1880],
        [-0.3994,  0.4889],
        [ 1.7330, -1.2958],
        [-3.5939,  0.2733],
        [ 0.4565, -0.8846],
        [-0.0637, -1.2511],
        [ 0.6540,  0.2579],
        [-0.6926, -0.8409],
        [ 0.3220, -0.0145],
        [ 0.3992, -0.0160],
        [-1.1754,  0.0604],
        [ 0.8875,  0.0444]])

In [16]:
# to get embedding of 5 value 
C[5]

tensor([ 0.5129, -0.5841])

In [19]:
emb = C[X]
emb.shape

torch.Size([32, 3, 2])

In [20]:
W1 = torch.randn ((6,100)) 
b1 = torch.randn (100)

In [30]:
emb @ W1 + b1

RuntimeError: mat1 and mat2 shapes cannot be multiplied (96x2 and 6x100)

In [24]:
# its not possible to direclty multipe W1 with emb vetor. So , we each emb vecotr and connecting with the other 
torch.cat([emb[:,0,:],emb[:,1,:],emb[:,2,:]],1).shape

torch.Size([32, 6])

In [25]:
# to make it dynamic on the blcok size 
torch.cat(torch.unbind(emb,1),1).shape

torch.Size([32, 6])

In [26]:
emb.view(32,6) == torch.cat(torch.unbind(emb,1),1)

tensor([[True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, T

In [33]:
h=emb.view(32,6) @ W1 +b1
h = emb.view(emb.shape[0],6 ) @ W1 + b1 #inorder to not hardcode the value of value 
h.shape

torch.Size([32, 100])

In [35]:
h = torch.tanh(emb.view(emb.shape[0],6) @ W1 +b1)
h

tensor([[-0.8936, -0.8340, -0.9217,  ..., -0.3022, -0.9039,  0.4577],
        [-0.7859, -0.7625, -0.6712,  ..., -0.2680, -0.7960,  0.8278],
        [-0.9199, -0.8500, -0.4453,  ...,  0.7303, -0.9572,  0.9928],
        ...,
        [-0.7706, -0.8062, -0.5810,  ...,  0.5779,  0.3040, -0.9999],
        [-0.9514,  0.6159,  0.2949,  ..., -0.9273,  0.8366, -0.9970],
        [-0.2808, -0.7293, -0.9167,  ..., -0.9932,  0.7251, -0.8787]])

In [36]:
W2 = torch.randn((100,27))
b2 = torch.randn(27)

In [38]:
logits = h @ W2 + b2 
logits.shape

torch.Size([32, 27])

In [39]:
counts = logits.exp()
prob = counts/ counts.sum(1,keepdim=True)


In [40]:
loss = - prob[torch.arange(32), Y ] .log().mean()
loss 

tensor(11.7466)

## Summary 

In [42]:
g= torch.Generator().manual_seed(2147483647)
C = torch.randn((27,2),generator=g) 
W1 = torch.randn ((6,100),generator=g)
b1 =torch.randn(100, generator=g)
W2 = torch.randn((100,27),generator=g) 
b2 = torch.randn (27,generator=g)

In [45]:
parameters = [C,W1,b1,W2,b2]
sum(p.nelement() for p in parameters)#number of parameters in total

3481

In [46]:
emb =C[X] #(32,3,2)
h = torch.tanh(emb.view(emb.shape[0],6)@ W1 +b1)#(32,100)
logits = h@W2+ b2
counts = logits.exp()
prob = counts/ counts.sum(1,keepdim=True)
loss = - prob[torch.arange(32),Y].log().mean()
loss

tensor(17.7697)